In [280]:
import pandas as pd
import matplotlib.pyplot as plt

import numpy as np

from bokeh.plotting import figure
from bokeh.models import Slider, ColumnDataSource, CDSView, IndexFilter, Plot, VBar, HoverTool, CustomJS, Select, Legend
from bokeh.layouts import row, gridplot, layout, column
from bokeh.io import show, save, curdoc
from bokeh.themes import Theme
from bokeh.palettes import viridis as palette
from bokeh.transform import factor_cmap

In [281]:
theme = Theme('../theme.yaml')
curdoc().theme = theme

In [282]:
df = pd.read_csv('sales_predictions_clean.csv')
outlets = df['Outlet_Identifier'].unique()
outlets= list(outlets)

In [284]:
### CREATING NEW DF to make CDS
outlet_filter = df['Outlet_Identifier'] == outlets[i]
outlet_Items = df[outlet_filter].drop(
    axis = 'columns', 
    columns=['Outlet_Identifier', 'Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type']
    )
outlet_Items['Item_Outlet_Sales'] = outlet_Items['Item_Outlet_Sales'].round()
outlet_Items['Units_Sold'] = (outlet_Items['Item_Outlet_Sales'] / outlet_Items['Item_MRP']).round()
outlet_Items['Item_Visibility_Adjusted'] = outlet_Items['Item_Visibility']*100
outlet_Items['Item_Visibility'] = outlet_Items['Item_Visibility'].round(5)
outlet_Items_CDS = ColumnDataSource(outlet_Items)

### STARTING BOKEH PLOT
fig_outlet_items_sales = figure(
    title=f'{outlets[i]} Total Sales',
    x_axis_label='Item MRP',
    y_axis_label='Total Sales USD'
    )

fig_outlet_items_sales.circle(
    x='Item_MRP', 
    y='Item_Outlet_Sales', 
    size='Item_Visibility_Adjusted',
    source=outlet_Items_CDS, 
    color=factor_cmap('Item_Type', palette= palette(len(outlet_Items.Item_Type.unique())),
    factors=outlet_Items.Item_Type.unique()),
    line_color='black',
    alpha=.75,
    muted_alpha=.2
    )

fig_outlet_items_sales.add_tools(HoverTool(
    tooltips=[
        ('Item', '@Item_Identifier'),
        ('Item Type', '@Item_Type'),
        ('Item Visibility', "@Item_Visibility"),
        ('Item MRP','@Item_MRP'), 
        ('Total Units Sold', '@Units_Sold'),
        ('Total Sales', '@Item_Outlet_Sales USD')
        ], 
    ))
plots.append(fig_outlet_items_sales)

layout = (gridplot(plots, ncols=2, toolbar_location= None, plot_height=300))
# show(layout)
# save(fig_outlet_items_sales, f'../plots/fig_{outlets[i]}_items_sales.html');
save(layout, f'../plots/outlets_items.html');

In [223]:
# df[Item_Type]
df['Item_Type'].value_counts().keys().tolist()

['Fruits and Vegetables',
 'Snack Foods',
 'Household',
 'Frozen Foods',
 'Dairy',
 'Canned',
 'Baking Goods',
 'Health and Hygiene',
 'Soft Drinks',
 'Meat',
 'Breads',
 'Hard Drinks',
 'Others',
 'Starchy Foods',
 'Breakfast',
 'Seafood']

In [88]:
# from bokeh.resources import CDN
# from bokeh.embed import components, autoload_static, file_html

# js, tag = components(fig_outlet_items_sales)
# # js, tag = autoload_static(fig_outlet_items_sales, CDN, 'fig_outlet_items_sales.js')
# # html = file_html(fig_outlet_items_sales, CDN, 'fig_outlet_item_sales')
# # html

# print(js)
# # print(js)
# print(tag)

# # CDN